<a href="https://colab.research.google.com/github/nikitamathewk/Predicting_Student_Personality/blob/main/prediction_mbti_Big_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
downloaded = drive.CreateFile({'id':'1-7pd4lcbnWV4b29P78bRdklqhg20jAOv'}) 
downloaded.GetContentFile('Emotion_Lexicon.csv') 
downloaded = drive.CreateFile({'id':'1dTN2A12rTCEKcD68Vb7v7GCdBx5VggWO'}) 
downloaded.GetContentFile('essays.csv') 
downloaded = drive.CreateFile({'id':'1sizqWKyKy1ls2J4Q5Yb8ruryAdHf21nG'}) 
downloaded.GetContentFile('mairesse.csv') 
downloaded = drive.CreateFile({'id':'1kynTRxSNIuBUNdlET7NyqP4coXSm6W7Q'}) 
downloaded.GetContentFile('mbti_1.csv') 
downloaded = drive.CreateFile({'id':'1J8ARJt1hC3FKQdfwaE5WXWq4P8okTUd1'}) 
downloaded.GetContentFile('glove.6B.300d.txt') 
downloaded = drive.CreateFile({'id':'1tR7zIjFSpd36bGX7mRszrHpC9-2byamO'}) 
downloaded.GetContentFile('glove.6B.50d.txt') 



In [4]:
import pandas as pd
import numpy as np
import pickle
import re

In [5]:
!mkdir data/essays/essays11142.p


mkdir: cannot create directory ‘data/essays/essays11142.p’: No such file or directory


In [6]:
!mkdir essays/essays_glove

mkdir: cannot create directory ‘essays/essays_glove’: No such file or directory


In [7]:

import re


class Essay:
    def __init__(self, text, cEXT, cNEU, cAGR, cCON, cOPN):
        self.text = text
        self.cEXT = cEXT
        self.cNEU = cNEU
        self.cAGR = cAGR
        self.cCON = cCON
        self.cOPN = cOPN
        self.clean_text = self.remove_unwanted_chars(text)
        self.words = self.get_all_words(text)
        self.scentences = self.create_scentences(text)
        self.filtered_text = ""
        self.w2v_words = []
        self.glove = {}

    
    # split long junks of words into scentences.
    
    def create_scentences(self, text):
        scentences = re.split("(?<=[.!?]) +", text)
        return scentences
    
    # all text is changed to lowercase 
    # and all caracters other than ASCII-letters, digits exclamation marks
    # and single and double quotation marks ,? are removed 
    
    def remove_unwanted_chars(self, text):
        allowed_chars = """ 0123456789abcdefghijklmnopqrstuvwxyz!"".?"""
        clean_text = text.lower()
        for c in clean_text:
            if allowed_chars.find(c) == -1:
                clean_text = clean_text.replace(c, "")
            else:
                pass
        return clean_text


    def get_all_words(self, text):
        
        regex = re.compile("""[^A-Za-z ]""")
        text = regex.sub('', text)
        text = text.split()
        
        return text

In [8]:
def mbti_to_big5(mbti):
   
  
    mbti = mbti.lower()
    cEXT, cNEU, cAGR, cCON, cOPN = 0,np.NaN,0,0,0
    if mbti[0] == "i":
        cEXT = 0
    elif mbti[0] == "e":
        cEXT = 1

    
    if mbti[1] == "n":
        cOPN = 1
    elif mbti[1] == "s":
        cOPN = 0   

    
    if mbti[2] == "t":
        cAGR = 0
    elif mbti[2] == "f":
        cAGR = 1

    if mbti[3] == "p":
        cCON = 0
    elif mbti[3] == "j":
        cCON = 1

    return cEXT, cNEU, cAGR, cCON, cOPN

In [9]:
def remove_unemotional_scentences(emotional_words, text_as_one_string):
    reduced_s = ""
    scentences = re.split('(?<=[.!?]) +', text_as_one_string)
    for s in scentences:
        
        if any(e in s for e in emotional_words):
            reduced_s = reduced_s + s + " "
        else:
            pass
    return reduced_s

In [10]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
#remove stopwords

def remove_stopwords(text_as_one_string):
  word_tokens = word_tokenize(text_as_one_string) 
  filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  filtered_sentence = [] 
  for w in word_tokens: 
    if w not in stop_words: 
        filtered_sentence.append(w) 
  


In [12]:
# every row of the read dataframe is put into a list of the object "Essay"
# remove data from list substract
def create_essays(df, subtract=None):
    essays = []
    for index, row in df.iterrows():
        essays.append(Essay(row.TEXT, row.cEXT, row.cNEU, row.cAGR, row.cCON, row.cOPN))  
    

    

    # removig scentences which do not contain emotionally charged words 
    
    if subtract != None:
        for x in essays:
            x.filtered_text = remove_stopwords(x.clean_text)
            x.filtered_text = remove_unemotional_scentences(emotional_words, x.clean_text)

    return essays

In [13]:
# reading the data from "essays.csv"  
df_essays = pd.read_csv('essays.csv', encoding='cp1252', delimiter=',', quotechar='"')

# for every essay, we replace the personalitiy categories of the essay wich are "y" and "n" with "1" and "0" 
for e in df_essays.columns[2:7]:
    df_essays[e] = df_essays[e].replace('n', '0')
    df_essays[e] = df_essays[e].replace('y', '1')
    
    df_essays[e] = pd.to_numeric(df_essays[e])

df_essays = df_essays[["TEXT", "cEXT", "cNEU", "cAGR", "cCON", "cOPN"]]
df_essays

,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,"Well, right now I just woke up from a mid-day ...",0,1,1,0,1
1,"Well, here we go with the stream of consciousn...",0,0,1,0,0
2,An open keyboard and buttons to push. The thin...,0,1,0,1,1
3,I can't believe it! It's really happening! M...,1,0,1,1,0
4,"Well, here I go with the good old stream of co...",1,0,1,0,1
...,...,...,...,...,...,...
2462,I'm home. wanted to go to bed but remembe...,0,1,0,1,0
2463,Stream of consiousnesssskdj. How do you s...,1,1,0,0,1
2464,"It is Wednesday, December 8th and a lot has be...",0,0,1,0,0
2465,"Man this week has been hellish. Anyways, now i...",0,1,0,0,1


In [14]:
#Load the MBTI kaggle dataset
df_kaggle = pd.read_csv('mbti_1.csv',  skiprows=0 )

df_kaggle["cEXT"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[0], 1)
df_kaggle["cNEU"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[1], 1)
df_kaggle["cAGR"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[2], 1)
df_kaggle["cCON"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[3], 1)
df_kaggle["cOPN"] =   df_kaggle.apply(lambda x: mbti_to_big5(x.type)[4], 1)

df_kaggle = df_kaggle[["posts", "cEXT", "cNEU", "cAGR", "cCON", "cOPN"]]
df_kaggle.columns = ["TEXT", "cEXT", "cNEU", "cAGR", "cCON", "cOPN"]

# remove |||
df_kaggle["TEXT"] = df_kaggle.apply(lambda x: x.TEXT.replace("|||", " ")[:], 1)


df_kaggle


,TEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,'http://www.youtube.com/watch?v=qsXHcwe3krw ht...,0,NaN,1,1,1
1,'I'm finding the lack of me in these posts ver...,1,NaN,0,0,1
2,'Good one _____ https://www.youtube.com/wat...,0,NaN,0,0,1
3,"'Dear INTP, I enjoyed our conversation the o...",0,NaN,0,1,1
4,'You're fired. That's another silly misconcept...,1,NaN,0,1,1
...,...,...,...,...,...,...
8670,'https://www.youtube.com/watch?v=t8edHB_h908 I...,0,NaN,1,0,0
8671,'So...if this thread already exists someplace ...,1,NaN,1,0,1
8672,'So many questions when i do these things. I ...,0,NaN,0,0,1
8673,'I am very conflicted right now when it comes ...,0,NaN,1,0,1


In [15]:
#"Emotional_Lexicon.csv" is a list of words and has several categories of emotions. 

df_lexicon = pd.read_csv('Emotion_Lexicon.csv', index_col=0)


# removing words  have no emotional category, 


df_lexicon = df_lexicon[(df_lexicon.T != 0).any()]
emotional_words = df_lexicon.index.tolist()

In [16]:
!mkdir essays

In [17]:
# save preprocessed data by converting into OBJECT essay and save with pickle and removing non emotional scentences
essays = create_essays(df_essays, emotional_words)
pickle.dump(essays, open( "essays/essays2467.p", "wb"))
print("saved entries: ", len(essays))

saved entries:  2467


In [18]:
#concatinate the dataframes:
frames  = [df_essays, df_kaggle]
essays_kaggle = pd.concat(frames, sort=False)
essays_kaggle.reset_index(drop=True)

## preprocess data by converting into OBJECT essay and save with pickle and removing non emotional scentences
essays_kaggle = create_essays(essays_kaggle, emotional_words)
pickle.dump(essays_kaggle, open("essays/essays11142.p", "wb"))
print("saved entries: ", len(essays_kaggle))

saved entries:  11142


#Global Vectors for Word Representation

In [19]:

import pickle
import numpy as np
import struct
import os
import pandas as pd

my_path = os.path.abspath(os.path.dirname('glove.6B.300d.txt'))
GLOVE_BIG = os.path.join(my_path, "glove.6B.300d.txt")
GLOVE_SMALL = os.path.join(my_path, "glove.6B.50d.txt")
encoding="utf-8"

In [20]:
essays =essays_kaggle

print("loaded count of essays:", len(essays))

loaded count of essays: 11142


In [21]:


class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        if len(word2vec)>0:
            self.dim=len(word2vec[next(iter(glove_mywords))])
        else:
            self.dim=0
            
    def fit(self, X, y):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [22]:
# load all vectors from all words from the GloVe File downladed from stanford
df = pd.read_csv(GLOVE_SMALL, sep=" ", quoting=3, header=None)

In [23]:
#load all words from all essays in a list
corpus = []
for e in essays:
    for w in e.words:
        corpus.append(w)
# and put it in a dataframe from this 
df_corpus = pd.DataFrame(corpus)
df_corpus

,0
0,Well
1,right
2,now
3,I
4,just
...,...
12496279,still
12496280,need
12496281,them
12496282,for


In [24]:
# inner join all GloVe Words with all words in the essays 
df_mywords = df.merge(df_corpus)
df_mywords = df_mywords.drop_duplicates()
df_mywords

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50
0,the,0.418000,0.249680,-0.412420,0.121700,0.345270,-0.044457,-0.496880,-0.178620,-0.000660,-0.656600,0.278430,-0.147670,-0.556770,0.146580,-0.00951,0.011658,0.102040,-0.127920,-0.84430,-0.121810,-0.016801,-0.332790,-0.155200,-0.231310,-0.191810,-1.88230,-0.76746,0.099051,-0.421250,-0.195260,4.0071,-0.185940,-0.522870,-0.31681,0.000592,0.007445,0.177780,-0.158970,0.012041,-0.054223,-0.298710,-0.157490,-0.347580,-0.045637,-0.44251,0.187850,0.002785,-0.184110,-0.115140,-0.785810
318030,of,0.708530,0.570880,-0.471600,0.180480,0.544490,0.726030,0.181570,-0.523930,0.103810,-0.175660,0.078852,-0.362160,-0.118290,-0.833360,0.11917,-0.166050,0.061555,-0.012719,-0.56623,0.013616,0.228510,-0.143960,-0.067549,-0.381570,-0.236980,-1.70370,-0.86692,-0.267040,-0.258900,0.176700,3.8676,-0.161300,-0.132730,-0.68881,0.184440,0.005246,-0.338740,-0.078956,0.241850,0.365760,-0.347270,0.284830,0.075693,-0.062178,-0.38988,0.229020,-0.216170,-0.225620,-0.093918,-0.803750
524784,to,0.680470,-0.039263,0.301860,-0.177920,0.429620,0.032246,-0.413760,0.132280,-0.298470,-0.085253,0.171180,0.224190,-0.100460,-0.436530,0.33418,0.678460,0.057204,-0.344480,-0.42785,-0.432750,0.559630,0.100320,0.186770,-0.268540,0.037334,-2.09320,0.22171,-0.398680,0.209120,-0.557250,3.8826,0.474660,-0.956580,-0.37788,0.208690,-0.327520,0.127510,0.088359,0.163510,-0.216340,-0.094375,0.018324,0.210480,-0.030880,-0.19722,0.082279,-0.094340,-0.073297,-0.064699,-0.260440
881734,and,0.268180,0.143460,-0.278770,0.016257,0.113840,0.699230,-0.513320,-0.473680,-0.330750,-0.138340,0.270200,0.309380,-0.450120,-0.412700,-0.09932,0.038085,0.029749,0.100760,-0.25058,-0.518180,0.345580,0.449220,0.487910,-0.080866,-0.101210,-1.37770,-0.10866,-0.232010,0.012839,-0.465080,3.8463,0.313620,0.136430,-0.52244,0.330200,0.337070,-0.356010,0.324310,0.120410,0.351200,-0.069043,0.368850,0.251680,-0.245170,0.25381,0.136700,-0.311780,-0.632100,-0.250280,-0.380970
1146979,in,0.330420,0.249950,-0.608740,0.109230,0.036372,0.151000,-0.550830,-0.074239,-0.092307,-0.328210,0.095980,-0.822690,-0.367170,-0.670090,0.42909,0.016496,-0.235730,0.128640,-1.09530,0.433340,0.570670,-0.103600,0.204220,0.078308,-0.427950,-1.79840,-0.27865,0.119540,-0.126890,0.031744,3.8631,-0.177860,-0.082434,-0.62698,0.264970,-0.057185,-0.073521,0.461030,0.308620,0.124980,-0.486090,-0.008027,0.031184,-0.365760,-0.42699,0.421640,-0.116660,-0.507030,-0.027273,-0.532850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10395620,frigging,-0.002708,-0.246300,0.336800,-0.742860,-0.462800,0.113000,0.520110,0.831020,0.548580,1.269800,-0.130060,-0.515580,0.038582,0.764830,0.55578,-0.224330,0.117360,0.174200,0.62166,-0.378780,-0.068022,0.391520,-0.519410,-0.290700,0.427920,1.11910,-0.54731,0.247020,1.650700,0.304450,-0.8041,0.563120,0.421980,0.86491,-0.175900,0.288510,0.457420,0.437200,-0.124590,-0.054119,-0.390020,-0.947280,0.447130,0.899360,-0.21329,-0.608260,0.131330,0.333040,-0.380220,0.331950
10395629,issie,-0.236050,-0.330200,-0.023831,0.101270,-0.391660,-0.242710,0.050236,0.549900,0.611950,0.303480,0.025502,0.495340,-0.076930,0.068972,-0.17817,0.131220,0.081281,-0.112910,1.20230,0.562450,-0.232940,0.184840,-0.144780,0.242150,0.991440,1.24720,-0.16548,0.164190,-0.193930,0.236190,-1.9715,-0.027835,0.975680,0.18562,-0.096861,-0.090310,-0.282170,0.363390,-0.316690,-0.312980,0.071789,-0.487380,-0.390720,-0.170260,-0.23974,-0.132530,0.326220,0.487550,0.177960,0.519250
10395630,etal,0.246830,-0.180910,0.087835,0.246700,-0.495280,0.052820,0.519070,0.235900,-0.359030,-0.208330,-0.357610,0.064963,-0.102670,-0.824710,0.37341,0.588140,0.291860,0.596870,0.62675,0.344400,0.434390,-0.195480,-0.776550,-0.017475,0.406640,1.16950,-0.21282,0.168630,0.054904,-0.000

In [25]:
#for the vectorizer we need a dict with all of our words
df_temp = df_mywords.set_index(0)
glove_mywords = {key: val.values for key, val in df_temp.T.items()}
glove_mywords

{'the': array([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
        -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
         2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
         1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
        -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
        -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
         4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
         7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
        -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
         1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01]),
 'of': array([ 0.70853  ,  0.57088  , -0.4716   ,  0.18048  ,  0.54449  ,
         0.72603  ,  0.18157  , -0.52393  ,  0.10381  , -0.17566  ,
         0.078852 , -0.36216  , -0.11829  , -0.83336  ,  0.11917  ,
        -0.16605  ,  0.061555 , -0.012719 , -0.5662

In [27]:
# for every essay we save the GloVe Vectors in essay.glove as a dictionary

for e in essays:
    df_temp_e = pd.DataFrame(e.words)
    try:
        
        df_temp_e = df_temp_e.merge(df_mywords)
        df_temp_e = df_temp_e.drop_duplicates()
        df_temp_e = df_temp_e.set_index(0)    
        e.glove = {key: val.values for key, val in df_temp_e.T.items()}
    except:
        print("error")

In [ ]:
!mkdir essays/essays

In [29]:
# save this essay data by converting into OBJECT essay and save with pickle and removing non emotional scentences
filename = "/content/essays/essays_glove" + "50" + "d_" + str(len(essays)) + ".p" 
pickle.dump(essays, open( filename, "wb"))
print("saved", len(essays), "entries: in", filename)

saved 11142 entries: in /content/essays/essays_glove50d_11142.p


#Split data in train & test

In [30]:
from sklearn.model_selection import train_test_split
training, test = train_test_split(essays, test_size=0.20, random_state=42)

In [34]:
train_x = [x.glove for x in training]

train_y_cEXT = [x.cEXT for x in training]
train_y_cNEU = [x.cNEU for x in training]
train_y_cAGR = [x.cAGR for x in training]
train_y_cCON = [x.cCON for x in training]
train_y_cOPN = [x.cOPN for x in training]


test_x = [x.glove for x in test]

test_y_cEXT = [x.cEXT for x in test]
test_y_cNEU = [x.cNEU for x in test]
test_y_cAGR = [x.cAGR for x in test]
test_y_cCON = [x.cCON for x in test]
test_y_cOPN = [x.cOPN for x in test]

train_x = np.array(train_x)
print(train_x)

train_y_cEXT = np.array(train_y_cEXT)
train_y_cNEU = np.array(train_y_cNEU)
train_y_cAGR = np.array(train_y_cAGR)
train_y_cCON = np.array(train_y_cCON)
train_y_cOPN = np.array(train_y_cOPN)

[{'recently': array([ 0.38306 , -0.18595 ,  0.15336 , -0.041891, -0.13426 ,  0.029778,
       -1.4633  , -0.33357 , -0.24152 , -0.12116 ,  0.25107 , -0.12706 ,
       -0.46931 , -0.1835  ,  0.66127 ,  0.32839 , -0.64247 , -0.017429,
       -0.103   ,  0.067737,  0.3842  ,  0.31524 ,  0.30089 , -0.29682 ,
       -0.40112 , -1.7654  ,  0.082136,  0.25844 , -0.6726  ,  0.16396 ,
        2.9298  , -0.35573 ,  0.19607 , -0.53139 , -0.08331 , -0.30637 ,
       -0.20509 ,  0.39256 , -0.16279 , -0.18471 , -0.41573 ,  0.36797 ,
       -0.02959 , -0.17756 ,  0.31373 , -0.17234 , -0.84572 ,  0.062888,
       -0.48442 ,  0.028268]), 'realized': array([ 0.53254 , -0.34176 ,  0.12317 , -0.61073 ,  0.73907 , -0.11109 ,
       -0.54485 ,  0.24482 , -0.42056 ,  0.34463 ,  0.30947 ,  0.44566 ,
       -0.5861  , -0.38819 ,  0.6012  ,  0.29977 ,  0.16197 ,  0.29134 ,
        0.054657, -0.40332 ,  0.083921,  0.49233 ,  0.13737 , -0.45364 ,
        1.2007  , -0.96364 , -0.90241 ,  0.041135,  0.6141  , -0.45

In [32]:
# (that's what they recommend on stanford for a simple approach) 
glove_vectorizer = MeanEmbeddingVectorizer(glove_mywords)

# create mean from our vectors

train_x_vectors = glove_vectorizer.transform(train_x)

test_x_vectors = glove_vectorizer.transform(test_x)

In [33]:
len(train_x_vectors)

8913

#SVM

In [36]:
from sklearn import svm
name = "svm"

print("training Extraversion cEXT using SVM...")
clf_svm_cEXT = svm.SVC(kernel='linear')
clf_svm_cEXT.fit(train_x_vectors, train_y_cEXT)

print("cEXT score: ", clf_svm_cEXT.score(test_x_vectors, test_y_cEXT))

try:
    print("training Neuroticism cNEU using SVM...")
    clf_svm_cNEU = svm.SVC(kernel='linear')
    clf_svm_cNEU.fit(train_x_vectors, train_y_cNEU)
   
    print("cNEU score: ", clf_svm_cNEU.score(test_x_vectors, test_y_cNEU))
except:
    print("with this data not available (MBTI only 4 dimensions)")
    
print("training Agreeableness cAGR using using SVM...")
clf_svm_cAGR = svm.SVC(kernel='linear')
clf_svm_cAGR.fit(train_x_vectors, train_y_cAGR)


print("cAGR score: ", clf_svm_cAGR.score(test_x_vectors, test_y_cAGR))

print("training Conscientiousness cCON using SVM...")
clf_svm_cCON = svm.SVC(kernel='linear')
clf_svm_cCON.fit(train_x_vectors, train_y_cCON)

print("cCON score: ", clf_svm_cCON.score(test_x_vectors, test_y_cCON))

print("training Openness to Experience cOPN using SVM...")
clf_svm_cOPN = svm.SVC(kernel='linear')
clf_svm_cOPN.fit(train_x_vectors, train_y_cOPN)

print("cOPN score: ", clf_svm_cOPN.score(test_x_vectors, test_y_cOPN))

training Extraversion cEXT using SVM...
cEXT score:  0.718259309107223
training Neuroticism cNEU using SVM...
with this data not available (MBTI only 4 dimensions)
training Agreeableness cAGR using using SVM...
cAGR score:  0.680574248541947
training Conscientiousness cCON using SVM...
cCON score:  0.582323912068192
training Openness to Experience cOPN using SVM...
cOPN score:  0.775235531628533


#Decision Tree

In [37]:
from sklearn import tree
name = "tree"

print("training Extraversion cEXT using dec...")
clf_dec_cEXT = tree.DecisionTreeClassifier()
clf_dec_cEXT.fit(train_x_vectors, train_y_cEXT)


print("cEXT score: ", clf_dec_cEXT.score(test_x_vectors, test_y_cEXT))

try:
    print("training Neuroticism cNEU using dec...")
    clf_dec_cNEU = tree.DecisionTreeClassifier()
    clf_dec_cNEU.fit(train_x_vectors, train_y_cNEU)
    
    print("cNEU score: ", clf_dec_cNEU.score(test_x_vectors, test_y_cNEU))
except:
    print("with this data not available (MBTI only 4 dimensions)")

print("training Agreeableness cAGR using using dec...")
clf_dec_cAGR = tree.DecisionTreeClassifier()
clf_dec_cAGR.fit(train_x_vectors, train_y_cAGR)

print("cAGR score: ", clf_dec_cAGR.score(test_x_vectors, test_y_cAGR))

print("training Conscientiousness cCON using dec...")
clf_dec_cCON = tree.DecisionTreeClassifier()
clf_dec_cCON.fit(train_x_vectors, train_y_cCON)

print("cCON score: ", clf_dec_cCON.score(test_x_vectors, test_y_cCON))

print("training Openness to Experience cOPN using dec...")
clf_dec_cOPN = tree.DecisionTreeClassifier()
clf_dec_cOPN.fit(train_x_vectors, train_y_cOPN)

print("cOPN score: ", clf_dec_cOPN.score(test_x_vectors, test_y_cOPN))

training Extraversion cEXT using dec...
cEXT score:  0.6096904441453567
training Neuroticism cNEU using dec...
with this data not available (MBTI only 4 dimensions)
training Agreeableness cAGR using using dec...
cAGR score:  0.5805293853746074
training Conscientiousness cCON using dec...
cCON score:  0.5338716913414087
training Openness to Experience cOPN using dec...
cOPN score:  0.7137729923732615


#Naive Bayes

In [38]:
from sklearn.naive_bayes import GaussianNB
name = "gNB"
# clf_gnb = GaussianNB()
# clf_gnb.fit(train_x_vectors, train_y)


print("training Extraversion cEXT using GaussianNaiveBayes...")
clf_gnb_cEXT = GaussianNB()
clf_gnb_cEXT.fit(train_x_vectors, train_y_cEXT)

print("cEXT score: ", clf_gnb_cEXT.score(test_x_vectors, test_y_cEXT))

try:
    print("training Neuroticism cNEU using GaussianNaiveBayes...")
    clf_gnb_cNEU = GaussianNB()
    clf_gnb_cNEU.fit(train_x_vectors, train_y_cNEU)
    
    print("cNEU score: ", clf_gnb_cNEU.score(test_x_vectors, test_y_cNEU))
except:
    print("with this data not available (MBTI only 4 dimensions)")

    
print("training Agreeableness cAGR using using GaussianNaiveBayes...")
clf_gnb_cAGR = GaussianNB()
clf_gnb_cAGR.fit(train_x_vectors, train_y_cAGR)

print("cAGR score: ", clf_gnb_cAGR.score(test_x_vectors, test_y_cAGR))

print("training Conscientiousness cCON using GaussianNaiveBayes...")
clf_gnb_cCON = GaussianNB()
clf_gnb_cCON.fit(train_x_vectors, train_y_cCON)

print("cCON score: ", clf_gnb_cCON.score(test_x_vectors, test_y_cCON))

print("training Openness to Experience cOPN using GaussianNaiveBayes...")
clf_gnb_cOPN = GaussianNB()
clf_gnb_cOPN.fit(train_x_vectors, train_y_cOPN)

print("cOPN score: ", clf_gnb_cOPN.score(test_x_vectors, test_y_cOPN))

training Extraversion cEXT using GaussianNaiveBayes...
cEXT score:  0.6864064602960969
training Neuroticism cNEU using GaussianNaiveBayes...
with this data not available (MBTI only 4 dimensions)
training Agreeableness cAGR using using GaussianNaiveBayes...
cAGR score:  0.6222521310004486
training Conscientiousness cCON using GaussianNaiveBayes...
cCON score:  0.5630327501121579
training Openness to Experience cOPN using GaussianNaiveBayes...
cOPN score:  0.759533423059668


#Logistic Regression

In [54]:
from sklearn.linear_model import LogisticRegression
name="logR"
print("training Extraversion cEXT using Logistic Regression...")
clf_log_cEXT = LogisticRegression(solver="newton-cg")
clf_log_cEXT.fit(train_x_vectors, train_y_cEXT)

print("cEXT score: ", clf_log_cEXT.score(test_x_vectors, test_y_cEXT))

try:
    print("training Neuroticism cNEU using Logistic Regression...")
    clf_log_cNEU = LogisticRegression(solver="newton-cg")
    clf_log_cNEU.fit(train_x_vectors, train_y_cNEU)
    
    print("cNEU score: ", clf_log_cNEU.score(test_x_vectors, test_y_cNEU))
except:
    print("with this data not available (MBTI only 4 dimensions)")
    
print("training Agreeableness cAGR using using Logistic Regression...")
clf_log_cAGR = LogisticRegression(solver="newton-cg")
clf_log_cAGR.fit(train_x_vectors, train_y_cAGR)

print("cAGR score: ", clf_log_cAGR.score(test_x_vectors, test_y_cAGR))

print("training Conscientiousness cCON using Logistic Regression...")
clf_log_cCON = LogisticRegression(solver="newton-cg")
clf_log_cCON.fit(train_x_vectors, train_y_cCON)

print("cCON score: ", clf_log_cCON.score(test_x_vectors, test_y_cCON))

print("training Openness to Experience cOPN using Logistic Regression...")
clf_log_cOPN = LogisticRegression(solver="newton-cg")
clf_log_cOPN.fit(train_x_vectors, train_y_cOPN)

print("cOPN score: ", clf_log_cOPN.score(test_x_vectors, test_y_cOPN))

training Extraversion cEXT using Logistic Regression...
cEXT score:  0.7196052041274114
training Neuroticism cNEU using Logistic Regression...
with this data not available (MBTI only 4 dimensions)
training Agreeableness cAGR using using Logistic Regression...
cAGR score:  0.6819201435621355
training Conscientiousness cCON using Logistic Regression...
cCON score:  0.6025123373710184
training Openness to Experience cOPN using Logistic Regression...
cOPN score:  0.8030506953790938


#Random Forest

In [40]:
from sklearn.ensemble import RandomForestClassifier
name="RF"


print("training Extraversion cEXT using Random Forest...")
clf_rf_cEXT = RandomForestClassifier(n_estimators=100)
clf_rf_cEXT.fit(train_x_vectors, train_y_cEXT)

print("cEXT score: ", clf_rf_cEXT.score(test_x_vectors, test_y_cEXT))

try:
    print("training Neuroticism cNEU using Random Forest...")
    clf_rf_cNEU = RandomForestClassifier(n_estimators=100)
    clf_rf_cNEU.fit(train_x_vectors, train_y_cNEU)
    
    print("cNEU score: ", clf_rf_cNEU.score(test_x_vectors, test_y_cNEU))
except:
    print("with this data not available (MBTI only 4 dimensions)")

print("training Agreeableness cAGR using using Random Forest...")
clf_rf_cAGR = RandomForestClassifier(n_estimators=100)
clf_rf_cAGR.fit(train_x_vectors, train_y_cAGR)

print("cAGR score: ", clf_rf_cAGR.score(test_x_vectors, test_y_cAGR))

print("training Conscientiousness cCON using Random Forest...")
clf_rf_cCON = RandomForestClassifier(n_estimators=100)
clf_rf_cCON.fit(train_x_vectors, train_y_cCON)

print("cCON score: ", clf_rf_cCON.score(test_x_vectors, test_y_cCON))

print("training Openness to Experience cOPN using Random Forest...")
clf_rf_cOPN = RandomForestClassifier(n_estimators=100)
clf_rf_cOPN.fit(train_x_vectors, train_y_cOPN)

print("cOPN score: ", clf_rf_cOPN.score(test_x_vectors, test_y_cOPN))

training Extraversion cEXT using Random Forest...
cEXT score:  0.7209510991475998
training Neuroticism cNEU using Random Forest...
with this data not available (MBTI only 4 dimensions)
training Agreeableness cAGR using using Random Forest...
cAGR score:  0.6693584567070435
training Conscientiousness cCON using Random Forest...
cCON score:  0.6047554957379991
training Openness to Experience cOPN using Random Forest...
cOPN score:  0.8061911170928667


#Predict

In [43]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import plotly.express as px
import pandas as pd
import re

In [44]:
#text = "Well, here we go with the stream of consciousness essay. I used to do things like this in high school sometimes. They were pretty interesting, but I often find myself with a lack of things to say. I normally consider myself someone who gets straight to the point. I wonder if I should hit enter any time to send this back to the front. Maybe I'll fix it later. My friend is playing guitar in my room now. Sort of playing anyway. More like messing with it. He's still learning. There's a drawing on the wall next to me. Comic book characters I think, but I'm not sure who they are. It's been a while since I've kept up with comic's. I just heard a sound from ICQ. That's a chat program on the internet. I don't know too much about it so I can't really explain too well. Anyway, I hope I'm done with this by the time another friend comes over. It will be nice to talk to her again. She went home this weekend for Labor Day. So did my brother. I didn't go. I'm not sure why. No reason to go, I guess. Hmm. when did I start this. Wow, that was a long line. I guess I won't change it later. Okay, I'm running out of things to talk about. I've found that happens to me a lot in conversation. Not a very interesting person, I guess. Well, I don't know. It's something I'm working on. I'm in a class now that might help. The phone just rang. Should I get it?  The guy playing the guitar answered it for me. It's for my roommate. My suitemate just came in and started reading this. I'm uncomfortable with that. He's in the bathroom now. You know, this is a really boring piece of literature. I never realized how dull most everyday thoughts are. Then again, when you keep your mind constantly moving like this, there isn't really time to stop and think deeply about things. I wonder how long this is going to be. I think it's been about ten minutes now. Only my second line. How sad. Well, not really considering how long these lines are. Anyway, I wonder what I'm going to do the rest of the night. I guess there's always homework to do. I guess we'll see. This seat is uncomfortable. My back sort of hurts. I think I'm going to have arthritis when I get older. I always thought that I wouldn't like to grow old. Not too old, I suppose. I've always been a very active person. I have a fear of growing old, I think. I guess it'll go away as I age gradually. I don't know how well I'd deal with paralysis from an accident though. As long as I have God and my friends around, I'll be okay though. I'm pretty thirsty right now. There isn't much to drink around my room. Ultimate Frisbee, I haven't played that all summer. Fun game, but tiring. I'm out of shape. I'd like to get in better shape, but I hate running. It's too dull for me. Hmmm. it's almost over now. Just a few more minutes. Let's see if I make it to the next line. Short reachable goals!  Whatever. Anyway, what else do I have to do tonight. I guess I could read some. My shirt smells like dinner. It's pretty disgusting. I need to wake up for a 9:30 am class tomorrow. I remember when that wasn't early at all. Well, I made it to the next line. I'm so proud of myself. That's sarcasm, by the way. I wonder if I was suppose to right this thing as a narrative. Oh well too late now. Time for me to head out. Until next time, good bye and good luck. I don't know. "

In [45]:
def create_essays_2(df, subtract=None):
    essays_2 = []
    
    if subtract != None:
        for x in essays:
            x.filtered_text = remove_stopwords(x.clean_text)
            x.filtered_text = remove_unemotional_scentences(emotional_words, x.clean_text)

    return essays_2

In [47]:
new_text_1 = pd.read_csv('/content/student.csv', encoding='cp1252', delimiter=',', quotechar='"')

In [56]:
def predict_personality(new_text_1):
    essays_2 = create_essays_2(new_text_1, emotional_words)
    essays_2_x = [[x.glove for x in essays_2]]
    #scentences = re.split("(?<=[.!?]) +", text)
    #essays_2= df_new_text_1.iloc[:, :2].values
    
 

    final_text = glove_vectorizer.transform(essays_2_x)




    EXT = clf_svm_cEXT.predict(final_text)
    NEU = clf_svm_cEXT.predict(final_text)
    AGR = clf_svm_cAGR.predict(final_text)
    CON = clf_svm_cCON.predict(final_text)
    OPN = clf_svm_cOPN.predict(final_text)
    #EXT = clf_log_cEXT.predict(final_text)
    #NEU = clf_log_cNEU.predict(final_text)
    #AGR = clf_log_cAGR.predict(final_text)
    #CON = clf_log_cCON.predict(final_text)
    #OPN = clf_log_cOPN.predict(final_text)
   
     
    return [EXT[0], NEU[0], AGR[0], CON[0], OPN[0]]

In [57]:

predictions = predict_personality(new_text_1)
print("predicted personality:", predictions)
df = pd.DataFrame(dict(r=predictions, theta=['EXT','NEU','AGR', 'CON', 'OPN']))
fig = px.line_polar(df, r='r', theta='theta', line_close=True)
fig.show()

predicted personality: [0, 0, 0, 0, 1]
